# 计算BOF和TDA所需要的时间

## 一些准备工作

### 一个计算运行时间的函数

In [31]:
import time

def coumpte_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function {func.__name__} took {execution_time} seconds to execute.")
        return result
    return wrapper


### 一个张量列表

In [32]:
import torch
# 创建一个空列表用于存放张量
tensor_list = []

# 定义张量形状
tensor_shape = (50000, 32*32*3)

# 生成5000个符合高斯分布的张量并添加到列表中
for _ in range(1):
    # 使用torch.randn生成符合标准正态分布的张量
    tensor = torch.randn(tensor_shape)
    tensor_list.append(tensor)

## 计算BOF消耗的时间

### 定义一个计算BOF的实例函数

In [33]:
from BOF.get_rank_from_matrix import Effective_Ranks
import torch



def get_BOF(tensor_list:list = []):
    # 现在tensor_list里面有5000个符合高斯分布的张量，每个张量形状为(5000, 32*32*3)
        

    results = []
    for image_matrix in tensor_list:
        # 检查张量形状
        if image_matrix.size(0) > image_matrix.size(1):
            image_matrix = image_matrix.T  # 如果 m > n，执行转置操作
        
        get_rank = Effective_Ranks(image_matrix)
        r0 = get_rank.r0
        R0 = get_rank.R0
        rk_max_index = get_rank.rk_max_index
        rk_max = get_rank.rk_max_value
        Rk_max = get_rank.Rk_value_max_rk_index

        results.append({"isic": {"r0": r0, "R0": R0, "rk_max_index": rk_max_index, "rk_max": rk_max, "Rk_max": Rk_max}})
    
    return results


In [34]:
# get_BOF()
# 使用function_A包装function_B
timed_function_B = coumpte_time(get_BOF)

# 调用timed_function_B，并传入参数
timed_function_B(tensor_list)  # 传入函数B所需的参数


torch.Size([3072, 50000]) torch.float32
(3072, 3072) float64
Function get_BOF took 8.636544466018677 seconds to execute.


[{'isic': {'r0': 1973.026815189157,
   'R0': 2894.0202108206254,
   'rk_max_index': 52,
   'rk_max': 2001.675971508667,
   'Rk_max': 2852.1083901756538}}]

## 计算TDA需要的时间

### 定义一个计算TDA的实例函数

In [35]:
from ripser import ripser

def get_TDA(tensor_list:list = []):

    # 现在tensor_list里面有5000个符合高斯分布的张量，每个张量形状为(5000, 32*32*3)
    
    results = []

    for image_matrix in tensor_list:
        TDA = ripser(X=image_matrix, maxdim=1)
        results.append(TDA['dgms'])

    return results
        

In [36]:
# get_BOF()
# 使用function_A包装function_B
timed_function_B = coumpte_time(get_TDA)

# 调用timed_function_B，并传入参数
timed_function_B(tensor_list)  # 传入函数B所需的参数


MemoryError: Unable to allocate 37.3 GiB for an array with shape (2499998288, 2) and data type int64

# 计算模型参数

## 定义一个计算参数数量的函数

In [37]:
def count_parameters(models):
    parameters_count = []

    for model in models:
        # print(type(model).__name__)
        parameters_count.append(sum(p.numel() for p in model.parameters()))

    return parameters_count

## 计算新ResNet

In [38]:
from nets.resnet import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
from nets.simple_net import MLP, LeNet

model_list = [MLP(), LeNet(), ResNet18(), ResNet34(), ResNet50(), ResNet101(), ResNet152()]

para_list = count_parameters(model_list)
print(para_list)

[1707274, 136886, 11173962, 21282122, 23520842, 42512970, 58156618]


## 计算旧ResNet

In [39]:
from nets.simple_net import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
from nets.simple_net import MLP, LeNet

model_list = [MLP(), LeNet(), ResNet18(), ResNet34(), ResNet50(), ResNet101(), ResNet152()]

para_list = count_parameters(model_list)
print(para_list)

[1707274, 136886, 11173962, 21282122, 21282122, 41353546, 57883978]
